In [1]:
! pip install evaluate
! pip install bert_score
! pip install sacrebleu
! pip install nltk
! pip install rouge_score
# for BLEURT refer to https://github.com/google-research/bleurt 


[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import json
import pprint 
import numpy as np
from evaluate import load

pp = pprint.PrettyPrinter(indent=4)
img_prompt_respond_file = '../data/memecap/filtered_meme_configs_5_attributes_memecap.json'
# Load predictions
predictions = []
with open(img_prompt_respond_file, 'r', encoding='utf-8') as json_file:
    predictions = json.load(json_file)
len(predictions), predictions[0]['meaning of the meme'], predictions[0]['image_dir'].split('/')[-1]

/home/shiling/anaconda3/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-12 11:34:01.211911: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-12 11:34:01.220523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 11:34:01.232282: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 11:34

(464,
 "- The meme humorously suggests that a wife, after losing her memory, fell in love with and decided to remarry her own husband, indicating his unchanging worth and likeability as a partner, paralleled by the character's proud declaration of worthiness regarding his hammer.",
 'memes_d079np.png')

In [3]:
# Load gtound truth 
gt = []
with open('../data/memecap/meme-cap-main/data/memes-test.json', 'r', encoding='utf-8') as json_file:
    gt = json.load(json_file)
len(gt), gt[0]['meme_captions'], gt[0]['img_fname']
memecap_predictions_concatenated = []
memecap_processed_gt_concatenated = []
memecap_predictions_extended = []
memecap_processed_gt_extended = []
for idx, meme_conf in enumerate(predictions):
    for meme in gt:
        if meme_conf['image_dir'].split('/')[-1] == meme['img_fname']:
            memecap_predictions_concatenated.append(meme_conf['meaning of the meme'])
            memecap_processed_gt_concatenated.append(''.join(meme['meme_captions']))
            
            memecap_predictions_extended += [meme_conf['meaning of the meme']] * len(meme['meme_captions'])
            memecap_processed_gt_extended += meme['meme_captions']
len(memecap_processed_gt_concatenated), len(memecap_processed_gt_extended), len(memecap_predictions_extended)

(464, 1567, 1567)

# BERTscore

In [4]:
bertscore = load("bertscore")

In [5]:
# Extended predictions
results = bertscore.compute(predictions=memecap_predictions_extended, references=memecap_processed_gt_extended, lang="en")
print(results.keys())
np.mean(results['precision']), np.mean(results['recall']), np.mean(results['f1'])

/home/shiling/anaconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


dict_keys(['precision', 'recall', 'f1', 'hashcode'])


(0.8596986209273566, 0.878484932191066, 0.8689020521791406)

In [6]:
# Concatenated references
results = bertscore.compute(predictions=memecap_predictions_concatenated, references=memecap_processed_gt_concatenated, lang="en")
print(results.keys())
np.mean(results['precision']), np.mean(results['recall']), np.mean(results['f1'])

dict_keys(['precision', 'recall', 'f1', 'hashcode'])


(0.8675926147863783, 0.8604510788773668, 0.8639449436089088)

# chrf

In [7]:
# prediction = ["The relationship between cats and dogs is not exactly friendly.", "a good bookshop is just a genteel black hole that knows how to read."]
# reference = [["The relationship between dogs and cats is not exactly friendly.", ], ["A good bookshop is just a genteel Black Hole that knows how to read."]]
chrf = load("chrf")

# {'score': 84.64214891738334, 'char_order': 6, 'word_order': 0, 'beta': 2}

In [8]:
# Extended predictions
chrf = load("chrf")
results = chrf.compute(predictions=memecap_predictions_extended, references=memecap_processed_gt_extended)
print(results)

{'score': 27.859525237388898, 'char_order': 6, 'word_order': 0, 'beta': 2}


In [9]:
# Concatenated references
results = chrf.compute(predictions=memecap_predictions_concatenated, references=memecap_processed_gt_concatenated)
print(results)

{'score': 23.778713464956684, 'char_order': 6, 'word_order': 0, 'beta': 2}


# bleurt

In [10]:
from bleurt import score

checkpoint = "/home/shiling/git/bleurt/BLEURT-20"
# [0.9881654977798462, 0.7926645278930664]
references = ["This is a test.", "This is a test."]
candidates = ["This is a test.", "This is the test."]

scorer = score.BleurtScorer(checkpoint)

INFO:tensorflow:Reading checkpoint /home/shiling/git/bleurt/BLEURT-20.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: /home/shiling/git/bleurt/BLEURT-20/sent_piece.model.
INFO:tensorflow:SentencePiece tokenizer created.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


I0000 00:00:1720776859.216812    8319 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1720776859.217434    8319 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1720776859.217511    8319 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1720776859.217972    8319 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [11]:
# Extended predictions
scores = scorer.score(references=memecap_processed_gt_extended, candidates=memecap_predictions_extended)
# assert isinstance(scores, list) and len(scores) == 1
print(np.mean(scores))

0.47095482013335105


In [12]:
# Concatenated references
scores = scorer.score(references=memecap_processed_gt_concatenated, candidates=memecap_predictions_concatenated)
# assert isinstance(scores, list) and len(scores) == 1
print(np.mean(scores))

0.47439709543411074


# rouge

In [13]:
rouge = load('rouge')
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]

# {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [14]:
# Extended predictions
results = rouge.compute(predictions=memecap_predictions_extended,
                        references=memecap_processed_gt_extended)
print(results)

{'rouge1': 0.21358210206311545, 'rouge2': 0.04168870174870554, 'rougeL': 0.17280080731314845, 'rougeLsum': 0.17281827452565934}


In [15]:
# Concatenated references
results = rouge.compute(predictions=memecap_predictions_concatenated,
                        references=memecap_processed_gt_concatenated)
print(results)

{'rouge1': 0.24106116976566622, 'rouge2': 0.05713127177377949, 'rougeL': 0.17311602335550197, 'rougeLsum': 0.17300583696935662}


# bleu

In [16]:
# predictions = ["hello there general kenobi", "foo bar foobar"]
# references = [
#     ["hello there general kenobi", "hello there !"],
#     ["foo bar foobar"]
# ]
bleu =load("bleu")

# {'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.1666666666666667, 'translation_length': 7, 'reference_length': 6}


In [21]:
# Extended predictions
bleu =load("bleu")
results = bleu.compute(predictions=memecap_predictions_extended, references=memecap_processed_gt_extended, max_order=4)
print(results)

{'bleu': 0.027211523349249925, 'precisions': [0.14962602704395564, 0.02523001043346296, 0.005337452099788848], 'brevity_penalty': 1.0, 'length_ratio': 2.0054679129567368, 'translation_length': 54282, 'reference_length': 27067}


In [18]:

# Concatenated references
results = bleu.compute(predictions=memecap_predictions_concatenated, references=memecap_processed_gt_concatenated)
print(results)

{'bleu': 0.018419506015368323, 'precisions': [0.31603215153592124, 0.06782162335579082, 0.01554130017855962, 0.005321689295217302], 'brevity_penalty': 0.5047994822571462, 'length_ratio': 0.5939674315321983, 'translation_length': 16049, 'reference_length': 27020}
